In [ ]:
#this notebook demonstrates baseis concepts in quantum chemsitry, namely effects of methods and basis sets

In [ ]:
#this cell initializes inbuilt functions

import numpy as np
import matplotlib.pyplot as plt
from pyscf import gto, scf, dft, mp, cc, tools
import ase
from ase import Atoms
from ase.build import molecule
from ase.visualize import view

from weas_widget import WeasWidget


def ase_to_pyscf(atoms, basis='sto-3g'):
    mol = gto.Mole()
    mol.atom = "\n".join(f"{atom.symbol} {pos[0]} {pos[1]} {pos[2]}" for atom, pos in zip(atoms, atoms.positions))
    mol.basis = basis
    mol.build()
    return mol


# Convert PySCF molecule to ASE Atoms object for visualization
def pyscf_to_ase(mol):
    symbols = [atom[0] for atom in mol.atom_coords()]  # Get element symbols
    positions = mol.atom_coords()  # Get atomic positions
    return Atoms(symbols=symbols, positions=positions)

In [ ]:
# Interactive ASE visualization and conversion back to PySCF
atoms = molecule("C6H6")  # Example molecule (benzene)
viewer = WeasWidget()
viewer.from_ase(atoms)
viewer

In [ ]:
#this box brings the molecule back into the work space, and sets up the calcuation

atoms = viewer.to_ase()
mol = ase_to_pyscf(atoms, 'sto-3g')



In [ ]:
#this box examines effects of different functionals

# Hartree-Fock Calculation
hf = scf.RHF(mol)
ehf = hf.kernel()
print(f"Hartree-Fock Energy: {ehf:.6f} Ha")

# DFT Calculation (B3LYP Functional)
dft_calc = dft.RKS(mol)
dft_calc.xc = 'b3lyp'
edft = dft_calc.kernel()
print(f"DFT (B3LYP) Energy: {edft:.6f} Ha")

# MP2 Calculation
mp2_calc = mp.MP2(hf)
emp2, _ = mp2_calc.kernel()
print(f"MP2 Correlation Energy: {emp2:.6f} Ha")
print(f"Total MP2 Energy: {ehf + emp2:.6f} Ha")

In [ ]:
# Basis Set Comparison
basis_sets = ['sto-3g', '6-31g', 'cc-pvdz']
energies = []
for basis in basis_sets:
    mol.basis = basis
    mol.build()
    hf = scf.RHF(mol).run()
    energies.append(hf.e_tot)

plt.figure()
plt.plot(basis_sets, energies, marker='o')
plt.xlabel('Basis Set')
plt.ylabel('Total Energy (Ha)')
plt.title('Basis Set Effect on Energy')
plt.show()